# 4. Theory and Mathematical Details

Interaction Primitives [1] are basic building blocks that model the movements between multiple agents in an interaction. This section will give a brief literature overview of the origins of Interaction Primitives before delving into a mathematical treatment of the three supported Interaction Primitive types, the: Interaction Probabilistic Movement Primitive, Bayesian Interaction Primitive, and ensemble Bayesian Interaction Primitive.

## Background

Inspired by Dynamical Movement Primitives [2], which were themselves inspired by the biological concept of Motor Primitives [3], Interaction Primitives probabilistically model the relationship multiple degrees of freedom distributed across two or more interacting agents. In the original formulation, Interaction Primitives were presented as a solution for exactly two agents, however, without loss of generality this formulation also applies to more than two agents (and in fact only one agent if some of the degrees of freedom are unobserved).

In both Dynamical Movement Primitives and Interaction Primitives, the trajectory of each degree of freedom is modeled as a nonlinear dynamical system. The parameters of the system -- specifically, the coefficients of the forcing function -- are determined from a set of demonstrations. In a Dynamic Movement Primitive, the system starts at some initial position and is pulled toward a specified goal position based on the attractor dynamics of the system. In this case, all degrees of freedom are observed at all times. Interaction Primitives extends this methodology by assuming some degrees of freedom are unknown (those associated with a controlled agent) and there is no known goal position. The system parameters associated with the unobserved degrees of freedom are estimated from a partial trajectory of the observed degrees of freedom (those associated with an observed agent).

While this is a powerful formulation, it falls short in one respect: it is non-trivial to combine multiple primitives together as building blocks in a more complex movement or interaction. While several solutions have been proposed to address this weakness, one of the most promising is the Probabilistic Movement Primitive [4]. In this formulation, the dynamical system has been eliminated entirely and replaced with a state space model composed of the parameterized degrees of freedom. Each degree of freedom is related to each other with a full joint probability distribution, allowing a well-defined probabilistic interpretation and the ability to easily combine and blend multiple primitives together, at the cost of the stability and convergence guarantees provided by the dynamical system. Such a formulation is easily extended to the interaction case (referred to as Interaction Probabilistic Movement Primitives) in which one or more degrees of freedom are unobserved [5].

One of the challenges that Interaction Primitives must deal with is the integration of a partial trajectory for the estimation of the unknown model parameters. In both Dynamic Movement Primitives and Probabilistic Movement Primitives, the parameterization of each degree of freedom is time-dependent. As such, the length of the partial trajectory must be estimated relative to the demonstration trajectories. In the standard formulation of Interaction Primitives, this estimation occurs as a separate sequence alignment step that is performed prior to parameter estimation. However, as this is a separate step, none of the uncertainty information associated with the model parameters is utilized. These are necessarily correlated since both the timing and shape of a trajectory are inherent properties of an interaction movement. Bayesian Interaction Primitives [6] were introduced as a solution, in which the length of the partial trajectory is simultaneously estimated with the model parameters in a fully probabilistic manner.

| Method | Model Type | Inference Type | Analysis Type |
| --- | --- | --- | --- |
| Interaction Primitives | Dynamical System | Spatial | Exact |
| Interaction Probabilistic Movement Primitive | Probabilistic | Spatial | Exact |
| Bayesian Interaction Primitives | Probabilistic | Spatiotemporal | Exact |
| Ensemble Bayesian Interaction Primitives | Probabilistic | Spatiotemporal | Approximate |

The above overview is high-level in nature as it is only intended to provide a background on the origins of Interaction Primitives. Of the listed algorithms, Interaction Probabilistic Movement Primitives, Bayesian Interaction Primitives, and ensemble Bayesian Interaction Primitives are supported by this library and so they will receive in an in-depth treatment in the following sections.

## Interaction Probabilistic Movement Primitive

We define an interaction $\boldsymbol{Y}$ as a time series of $D$-dimensional sensor observations over time, $\boldsymbol{Y}_{1:T} = [\boldsymbol{y}_1, \dots, \boldsymbol{y}_T] \in \mathbb{R}^{D \times T}$. Of the $D$ dimensions, $D_o$ of them represent \emph{observed} DoFs from one agent (the human) and $D_c$ of them represent the \emph{controlled} DoFs from the other agent (the robot), such that $D = D_c + D_o$.

### Basis Function Decomposition

Rather than working directly with the time series, the interaction $\boldsymbol{Y}$ is represented with a state space model.
With a standard parameterization where the number of state variables is equivalent to the number of degrees of freedom, the shape of the trajectory is captured through non-trivial state transition dynamics.
This is undesirable, as the trajectory shape needs to be modifiable online based on observations, which is difficult to accomplish if the shape is modeled solely by the transition dynamics.
Instead, we transform the interaction $\boldsymbol{Y}$ into a latent space via basis function decomposition such that the trajectory shape is modeled as part of the state itself.
With such a model, the shape of the trajectory -- including the goal the point -- can be adjusted by simply updating the state estimate.

Each dimension $d \in D$ of $\boldsymbol{Y}$ is approximated with a weighted linear combination of time-dependent nonlinear basis functions, such that $y_t^d = h^d(\phi(t), \boldsymbol{w}^d) = \Phi_{\phi(t)}^{\intercal} \boldsymbol{w}^d + \epsilon_y$, where $\Phi_{\phi(t)} \in \mathbb{R}^{1\times B}$ is a row vector of $B^d$ basis functions, $\boldsymbol{w}^d \in \mathbb{R}^{B \times 1}$, and $\epsilon_y$ is i.i.d. Gaussian noise.
As this is a linear system with a closed-form solution, the weights $\boldsymbol{w}^d$ can be found through simple linear regression, i.e., least squares.
The full latent model is composed of the aggregated weights from each dimension, $\boldsymbol{w} = [\boldsymbol{w}^{1\intercal}, \dots, \boldsymbol{w}^{D\intercal}] \in \mathbb{R}^{1 \times B}$ where $B = \sum_{d}^{D} B^d$ and $\boldsymbol{y}_t = h(\phi(t), \boldsymbol{w})$.

We note that the time-dependence of the basis functions -- and the nonlinear function $h(\cdot)$ -- is not on the absolute time $t$, but rather on a relative phase value $\phi(t)$.
Consider the basis function decompositions for a motion performed at slow speeds and fast speeds with a fixed measurement rate.
If the time-dependence is based on the absolute time $t$, then the decompositions will be different despite the motion being spatially identical.
Thus, we substitute the absolute time $t$ with a linearly interpolated relative phase value, $\phi(t)$, such that $\phi(0) = 0$ and $\phi(T) = 1$.
For notational simplicity, from here on we refer to $\phi(t)$ as simply $\phi$.


### Temporal Estimation via Sequence Alignment

Given $t$ observations of an interaction, $\boldsymbol{Y}_{1:t}$, the goal of Interaction Primitives is to infer the underlying latent model $\boldsymbol{w}$ while taking into account a prior model $\boldsymbol{w}_0$.
We assume that the $t$ observations made so far are of a partial interaction, i.e., $\phi(t) < 1$, and that $T$ is unknown.
Formally, we define the solution to this problem as the following posterior distribution:

(1)
$$
\begin{equation*}
    p(\boldsymbol{w}_t | \boldsymbol{Y}_{1:t}, \boldsymbol{w}_{0}) \propto p(\boldsymbol{y}_{t} | \boldsymbol{w}_t) p(\boldsymbol{w}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{w}_{0}).
\end{equation*}
$$

As the latent state space utilizes phase-dependent basis functions, we must estimate $\phi(t)$ before we can update our state estimate with the partial observations.
In Interaction Probabilistic Movement Primitives, this is accomplished with the application of a sequence alignment algorithm -- Dynamic Time Warping -- against the demonstration trajectories.

Given the mean demonstration trajectory $\boldsymbol{\bar{Y}}$ of length $M$ and the partially observed trajectory $\boldsymbol{Y}$ of length $T$, Dynamic Time Warping computes the optimal alignment which minimizes a distance function $c(\cdot)$ (typically Euclidean) as follows.

$$
\begin{align*}
D(1,t) &= \sum_{k=1}^{t} c(\boldsymbol{\bar{y}}_1, \boldsymbol{y}_t), m \in [1 : T], \\
D(m,1) &= \sum_{k=1}^{m} c(\boldsymbol{\bar{y}}_k, \boldsymbol{y}_1), t \in [1 : M], \\
D(m,t) &= \text{min}[D(m-1,t-1), D(m,t-1), D(m-1,t)] + c(\boldsymbol{\bar{y}}_m, \boldsymbol{y}_t)
\end{align*}
$$

This recursion can be terminated upon finding the distance to the pair $(m*,T)$ where $m* = \text{argmin}_{m} D(m,T)$. The estimated phase is then given by the ratio $\phi = \frac{m*}{M}$.


### Exact Spatial Inference

The posterior density in Eq. 1 is computed with a recursive linear state space filter, i.e., a standard Kalman filter.
Such filters are composed of two steps performed recursively: state prediction in which the state is propagated forward in time according to the system dynamics $p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{s}_{0})$, and measurement update in which the latest sensor observation is incorporated in the predicted state $p(\boldsymbol{y}_{t} | \boldsymbol{s}_t)$.
Applying Markov assumptions, the state prediction density can be defined as:

$$
\begin{align*}
    & p(\boldsymbol{w}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{w}_{0}) \nonumber \\
    & = \int p(\boldsymbol{w}_t | \boldsymbol{w}_{t-1}) p(\boldsymbol{w}_{t-1} | \boldsymbol{Y}_{1:t-1}, \boldsymbol{w}_{0})d\boldsymbol{w}_{t-1}.
\end{align*}
$$

As with all Kalman filters, we assume that all error estimates produced during recursion are normally distributed, i.e., $p(\boldsymbol{w}_t | \boldsymbol{Y}_{1:t}, \boldsymbol{w}_{0}) = \mathcal{N}(\boldsymbol{\mu}_{t|t}, \boldsymbol{\Sigma}_{t|t})$ and $p(\boldsymbol{w}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{w}_{0}) = \mathcal{N}(\boldsymbol{\mu}_{t|t-1}, \boldsymbol{\Sigma}_{t|t-1})$.
The state itself is time-invariant, therefore the state transition matrix $\boldsymbol{G}$ is simply the identity

$$
\begin{align*}
    \boldsymbol{\mu}_{t|t-1} &= 
    {\underbrace{
            \begin{bmatrix}
            1 & \dots & 0\\
            \vdots & \ddots & \vdots\\
            0 & \dots & 1
            \end{bmatrix}
        }_\text{$\boldsymbol{G}$}}
    \boldsymbol{\mu}_{t-1|t-1}, \\
    \boldsymbol{\Sigma}_{t|t-1} &= \boldsymbol{G} \boldsymbol{\Sigma}_{t-1|t-1} \boldsymbol{G}^{\intercal} +
    \underbrace{\begin{bmatrix}
        1 & \dots & 0\\
        \vdots & \ddots & \vdots\\
        0 & \dots & 1
        \end{bmatrix}}_\text{$\boldsymbol{Q}_t$},
\end{align*}
$$

and so is $\boldsymbol{Q}$, the process noise associated with the state transition update.

The observation function $h(\cdot)$ is linear with respect to the state variable $\boldsymbol{w}$ which yields a straightforward observation matrix $\boldsymbol{H}$:

$$
\begin{align*}
    \begin{split}
    \boldsymbol{H}_t &= \frac{\partial h(\boldsymbol{w}_t)}{\partial h_t}\\
    &= 
    \begin{bmatrix}
    \Phi_{\phi} & \dots & 0\\
    \vdots & \ddots & \vdots\\
    0 & \dots & \Phi_{\phi}
    \end{bmatrix}.
    \end{split}
\end{align*}
$$

We can integrate the measurement by calculating the innovation covariance as well as the Kalman gain, which dictates how heavily the observation should be weighted:

$$
\begin{align*}
    \boldsymbol{S}_t &= \boldsymbol{H}_t \boldsymbol{\Sigma}_{t|t-1} \boldsymbol{H}_t^{\intercal} + \boldsymbol{R}_t,\\
    \boldsymbol{K}_t &= \boldsymbol{\Sigma}_{t|t-1} \boldsymbol{H}_t^{\intercal} \boldsymbol{S}_t^{-1}.
\end{align*}
$$

This enables the calculation of the parameters for the posterior distribution,

$$
\begin{align*}
    \boldsymbol{\mu}_{t|t} &= \boldsymbol{\mu}_{t|t-1} + \boldsymbol{K}_t(\boldsymbol{y}_t - h(\boldsymbol{\mu}_{t|t-1})),\\
    \boldsymbol{\Sigma}_{t|t} &= (I - \boldsymbol{K}_t \boldsymbol{H}_t)\boldsymbol{\Sigma}_{t|t-1},
\end{align*}
$$

where $\boldsymbol{R}_t$ is the Gaussian measurement noise associated with the sensor observation $\boldsymbol{y}_t$.



The prior model $\boldsymbol{w}_0$ is computed from a set of initial demonstrations.
That is, given the latent models for $N$ demonstrations, $\boldsymbol{W} = [\boldsymbol{w}_1^{\intercal}, \dots, \boldsymbol{w}_N^{\intercal}]$, we define $\boldsymbol{w}_0$ as simply the arithmetic mean of each DoF:

$$
\begin{equation*}
    \boldsymbol{w}_0 = \left[\frac{1}{N}\sum_{i=1}^{N}\boldsymbol{w}^1_i, \dots, \frac{1}{N}\sum_{i=1}^{N}\boldsymbol{w}^D_i\right].
\end{equation*}
$$

where $T_i$ is the length of the $i$-th demonstration.
The prior density is defined as $p(\boldsymbol{w}_0) = \mathcal{N}(\boldsymbol{\mu}_0, \boldsymbol{\Sigma}_0)$ where

$$
\begin{align*}
    \boldsymbol{\mu}_0 &= \boldsymbol{w}_0,\\
    \boldsymbol{\Sigma}_0 &= \boldsymbol{\Sigma}_{\boldsymbol{W}, \boldsymbol{W}}
\end{align*}
$$

and $\boldsymbol{\Sigma}_{\boldsymbol{W}, \boldsymbol{W}}$ is the sample covariance of the basis weights.

The baseline measurement noise $\boldsymbol{R}$ is calculated from the set of initial demonstrations with the following closed-form solution:

$$
\begin{align*}
    \boldsymbol{R} = \frac{1}{N} \sum_{i}^{N} \frac{1}{T_i} \sum_{t}^{T_i} \left( \boldsymbol{y}_{t} - h(\phi(t), \boldsymbol{w}_i) \right)^{2}.
\end{align*}
$$

This value is equivalent to the mean squared error of the regression fit for our basis functions over every demonstration.
Intuitively, this represents the variance of the data around the regression and captures both the approximation error and the sensor noise associated with the observations.


## Bayesian Interaction Primitives

Bayesian Interaction Primitives are based upon the observation that the temporal and spatial estimation problems are inherently correlated, and that we are throwing information away by solving only one problem at a time in isolation.
In other words, if we mis-estimate where we are in the interaction in a temporal sense, we will mis-estimate where we are in a spatial sense as well.
This is based upon insights from the simultaneous localization and mapping (SLAM) field, where spatial errors in the robot's state estimate result in correlated errors in each map landmark.

Intuitively, this can be understood by imagining that we are wearing a blindfold and walking around a room filled with a set of landmarks, such as walls and tables and chairs.
Furthermore, we are allowed to periodically remove our blindfold and make an observation.
As we move around while blindfolded, we maintain an estimate of where we think we are (our localized state) relative to each landmark in the room (the map).
Suppose we remove our blindfold after moving and find that a landmark is not where we expected it to be.
If this is the only landmark we know of in the room, then it is difficult to determine the source of the error: is it due to a bad initial observation of the landmark, i.e., a noisy measurement?
Or is it due to us being wrong about where we think we are with respect to the landmark, i.e., our state estimate?

Now suppose there are five landmarks in the room.
If four of the landmarks are exactly where we thought they would be, but the fifth one isn't, then the source of the error is much more likely to be a noisy observation of the fifth landmark rather than an error in our state (as that would require noisy observations of the other four landmarks to coincidentally be correct).
But what if all five landmarks are in different positions than we expected?
If the source of the error is our state estimate, then we would expect there to be some sort of relationship between the errors of the landmarks.
Thus we can succinctly describe this dual optimization problem as follows: an error in the state estimate induces a correlated error in the landmark estimates.

This same scenario holds when considering interactions, only in this case our state estimate is our temporal location, the phase $\phi$, and the landmarks are the weights of our decomposed DoF trajectories, $\boldsymbol{w}$.
The above explanation was rather simple for illustrative purposes, as there are many other factors to consider, for example how accurate we estimate our observations to inherently be, how accurate we expect our state estimate to be over time, etc.
Complicating this even more is the fact that our "map" is not static!
People can often be unpredictable and they may change their trajectories at any point in an interaction.
So even if our estimate of the weights of the interaction (our "map") is correct at one point in time, this does not guarantee that it will be in the future.
In the SLAM example, this would be as if the tables and chairs would move about on their own while we are blindfolded; or if the walls take a page out of `Inception` and start changing their shape.


### Exact Spatiotemporal Inference

We formulate this spatiotemporal inference problem much the same as before, except our state vector must be expanded to account for the temporal terms.
Probabilistically, we represent this insight with the augmented state vector $\boldsymbol{s} = [\phi, \dot{\phi}, \boldsymbol{w}]$ and the following definition:

$$
\begin{equation*}
    p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t}, \boldsymbol{s}_{0}) \propto p(\boldsymbol{y}_{t} | \boldsymbol{s}_t) p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{s}_{0}).
\end{equation*}
$$

It is important to note that while the weights themselves are time-invariant with respect to an interaction, our estimate of the weights \emph{is} time-varying.
That is, every time we integrate a new sensor observation, our estimate of the underlying latent model is updated.

As before, we compute the posterior with a two-step recursive filter: state prediction in which the state is propagated forward in time according to the system dynamics $p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{s}_{0})$, and measurement update in which the latest sensor observation is incorporated in the predicted state $p(\boldsymbol{y}_{t} | \boldsymbol{s}_t)$.
Applying Markov assumptions, the state prediction density can be defined as:

$$
\begin{align*}
    & p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{s}_{0}) \nonumber \\
    & = \int p(\boldsymbol{s}_t | \boldsymbol{s}_{t-1})
    p(\boldsymbol{s}_{t-1} | \boldsymbol{Y}_{1:t-1}, \boldsymbol{s}_{0})d\boldsymbol{s}_{t-1}.
\end{align*}
$$

Again, we assume that all error estimates produced during recursion are normally distributed, i.e., $p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t}, \boldsymbol{s}_{0}) = \mathcal{N}(\boldsymbol{\mu}_{t|t}, \boldsymbol{\Sigma}_{t|t})$ and $p(\boldsymbol{s}_t | \boldsymbol{Y}_{1:t-1}, \boldsymbol{s}_{0}) = \mathcal{N}(\boldsymbol{\mu}_{t|t-1}, \boldsymbol{\Sigma}_{t|t-1})$.
Unlike before, however, the temporal terms of our state do evolve over time.
For simplicity, we assume that the state evolves according to a linear constant velocity model:

(2)
$$
\begin{align*}
    \boldsymbol{\mu}_{t|t-1} &= 
    {\underbrace{
            \begin{bmatrix}
            1 & \Delta t & \dots & 0\\
            0 & 1 & \dots & 0\\
            \vdots & \vdots & \ddots & \vdots\\
            0 & 0 & \dots & 1
            \end{bmatrix}
        }_\text{$\boldsymbol{G}$}}
    \boldsymbol{\mu}_{t-1|t-1}, \\
    \boldsymbol{\Sigma}_{t|t-1} &= \boldsymbol{G} \boldsymbol{\Sigma}_{t-1|t-1} \boldsymbol{G}^{\intercal} +
    \underbrace{\begin{bmatrix}
        \boldsymbol{\Sigma}_{\phi, \dot{\phi}} & \dots & 0\\
        \vdots & \ddots & \vdots\\
        0 & \dots & 1
        \end{bmatrix}}_\text{$\boldsymbol{Q}_t$}
\end{align*}
$$

where $\boldsymbol{Q}$ is the process noise associated with the state transition update.
The noise correlations between phase and phase velocity, $\boldsymbol{\Sigma}_{\phi, \dot{\phi}}$, are determined by a piecewise or continuous first-order white noise model, e.g.,

$$
\begin{align*}
    \boldsymbol{\Sigma}_{\phi, \dot{\phi}} = 
    \begin{bmatrix}
    	\frac{\Delta t^4}{4} & \frac{\Delta t^3}{3}\\
    	\frac{\Delta t^3}{3} & \Delta t^2
	\end{bmatrix} \sigma^2_\phi. \nonumber
\end{align*}
$$

Unlike in Interaction ProMP, the observation function $h(\cdot)$ is now nonlinear with respect to the state variable $\phi$ and must be linearized via Taylor expansion:

(3)
$$
\begin{align*}
    \begin{split}
    \boldsymbol{H}_t &= \frac{\partial h(\boldsymbol{s}_t)}{\partial s_t}\\
    &= 
    \begin{bmatrix}
    \frac{\partial \Phi_{\phi}^{\intercal} \boldsymbol{w}^1}{\partial \phi} & 0 & \Phi_{\phi} & \dots & 0\\
    \vdots & \vdots & \vdots & \ddots & \vdots\\
    \frac{\partial \Phi_{\phi}^{\intercal} \boldsymbol{w}^{D}}{\partial \phi} & 0 & 0 & \dots & \Phi_{\phi}
    \end{bmatrix}.
    \end{split}
\end{align*}
$$

Note that because the augmented state now includes the phase $\phi$, the observation function $h(\boldsymbol{s})$ is simply a function of $\boldsymbol{s}$ in order to reduce notational clutter.
We can now integrate the measurement by calculating the innovation covariance as well as the Kalman gain, which dictates how heavily the observation should be weighted:

$$
\begin{align*}
    \boldsymbol{S}_t &= \boldsymbol{H}_t \boldsymbol{\Sigma}_{t|t-1} \boldsymbol{H}_t^{\intercal} + \boldsymbol{R}_t,\\
    \boldsymbol{K}_t &= \boldsymbol{\Sigma}_{t|t-1} \boldsymbol{H}_t^{\intercal} \boldsymbol{S}_t^{-1}.
\end{align*}
$$

This enables the calculation of the parameters for the posterior distribution,

(4)
$$
\begin{align*}
    \boldsymbol{\mu}_{t|t} &= \boldsymbol{\mu}_{t|t-1} + \boldsymbol{K}_t(\boldsymbol{y}_t - h(\boldsymbol{\mu}_{t|t-1})),\\
    \boldsymbol{\Sigma}_{t|t} &= (I - \boldsymbol{K}_t \boldsymbol{H}_t)\boldsymbol{\Sigma}_{t|t-1},
\end{align*}
$$

where $\boldsymbol{R}_t$ is the Gaussian measurement noise associated with the sensor observation $\boldsymbol{y}_t$.



The prior model $\boldsymbol{s}_0 = [\phi_0, \dot{\phi}_0, \boldsymbol{w}_0]$ is computed from a set of initial demonstrations.
That is, given the latent models for $N$ demonstrations, $\boldsymbol{W} = [\boldsymbol{w}_1^{\intercal}, \dots, \boldsymbol{w}_N^{\intercal}]$, we define $\boldsymbol{w}_0$ as simply the arithmetic mean of each DoF:

$$
\begin{equation*}
    \boldsymbol{w}_0 = \left[\frac{1}{N}\sum_{i=1}^{N}\boldsymbol{w}^1_i, \dots, \frac{1}{N}\sum_{i=1}^{N}\boldsymbol{w}^D_i\right].
\end{equation*}
$$

The initial phase $\phi_0$ is set to $0$ under the assumption that all interactions start from the beginning.
The initial phase velocity $\dot{\phi}_0$ is the arithmetic mean of the phase velocity of each demonstration:

$$
\begin{equation*}
    \dot{\phi}_0 = \frac{1}{N} \sum_{i=1}^N \frac{1}{T_i},
\end{equation*}
$$

where $T_i$ is the length of the $i$-th demonstration.
The prior density is defined as $p(\boldsymbol{s}_0) = \mathcal{N}(\boldsymbol{\mu}_0, \boldsymbol{\Sigma}_0)$ where

(5)
$$
\begin{align*}
    \boldsymbol{\mu}_0 &= \boldsymbol{s}_0, \\
    \boldsymbol{\Sigma}_0 &=
    \begin{bmatrix}
    \boldsymbol{\Sigma}_{\phi, \phi} & 0 & 0\\
    0 & \boldsymbol{\Sigma}_{\dot{\phi}, \dot{\phi}} & 0\\
    0 & 0 & \boldsymbol{\Sigma}_{\boldsymbol{W}, \boldsymbol{W}}
    \end{bmatrix},
\end{align*}
$$

and $\boldsymbol{\Sigma}_{\phi, \phi}$ is the sample variance of the phases of the demonstrations, $\boldsymbol{\Sigma}_{\dot{\phi}, \dot{\phi}}$ is the sample variance of the phase velocities, and $\boldsymbol{\Sigma}_{\boldsymbol{W}, \boldsymbol{W}}$ is the sample covariance of the basis weights.

The measurement noise $\boldsymbol{R}$ is calculated in the same manner as before.

## Ensemble Bayesian Interaction Primitives

While Bayesian Interaction Primitives is an analytical method to compute the exact solution to the simultaneous temporal and spatial estimation problem, it suffers from several drawbacks in practice: 1) the prior distribution is assumed to be Gaussian, which is unlikely to be the case; 2) the first-order Taylor approximation introduces (potentially significant) linearization errors; and 3) the integration of the covariance matrix is computationally prohibitive for large state dimensions.
Therefore, in addition to the exact solution yielded by Bayesian Interaction Primitives, we also present a Monte Carlo-based method known as ensemble Bayesian Interaction Primitives (eBIP).
Originally motivated as a solution to multimodal applications which amplify the above problems [7], this method also yields improvements in inference accuracy and computational performance in the general case.

### Approximate Spatiotemporal Inference

The extended Kalman filter employed for recursive filtering in BIP relies on the assumption that uncertainty in the state prediction is approximately Gaussian.
When this is not the case, the estimated state can diverge rapidly from the true state.
One potential source of non-normality in the uncertainty is the nonlinear state transition or observation function in the dynamical system.
The original formulation of BIP addresses this challenge by linearizing these functions about the estimated state via first-order Taylor approximation, which is performed in Eq. 3 for the nonlinear observation function $h(\cdot)$.
Unfortunately, this produces linearization errors resulting from the loss of information related to higher-order moments.
In strongly nonlinear systems this can result in poor state estimates and in the worst case cause divergence from the true state.

We follow an ensemble-based filtering methodology [8] which avoids the Taylor series approximation and hence the associated linearization errors. Fundamentally, we approximate the state prediction with a Monte Carlo approximation where the sample mean of the ensemble models the mean $\boldsymbol{\mu}$ and the sample covariance the covariance $\boldsymbol{\Sigma}$.
Thus, rather than calculating these values explicitly during state prediction at time $t$ as in Eq. 4, we instead start with an ensemble of $E$ members sampled from the prior distribution $\mathcal{N}(\boldsymbol{\mu}_{t-1|t-1}, \boldsymbol{\Sigma}_{t-1|t-1})$ such that $\boldsymbol{X}_{t-1|t-1} = [\boldsymbol{x}^1,\dots,\boldsymbol{x}^E]$.
Each member is propagated forward in time using the state evolution model with an additional perturbation sampled from the process noise,

$$
\begin{align*}
    \boldsymbol{x}^j_{t|t-1} &=
    \boldsymbol{G} \boldsymbol{x}^j_{t-1|t-1}
    +
    \mathcal{N}
    \left(0, \boldsymbol{Q}_t\right), \quad 1 \leq j \leq E.
\end{align*}
$$


As $E$ approaches infinity, the ensemble effectively models the full covariance calculated in Eq. 2 [8].
We note that in BIP the state transition function is linear, however, when this is not the case the nonlinear function $g(\cdot)$ is used directly.

During the measurement update step, we calculate the innovation covariance $\boldsymbol{S}$ and the Kalman gain $\boldsymbol{K}$ directly from the ensemble, with no need to specifically maintain a covariance matrix.
We begin by calculating the transformation of the ensemble to the measurement space, via the nonlinear observation function $h(\cdot)$, along with the deviation of each ensemble member from the sample mean:

$$
\begin{align*}
    \boldsymbol{H}_t\boldsymbol{X}_{t|t-1} &= \left[h(\boldsymbol{x}^1_{t|t-1}), \dots, h(\boldsymbol{x}^E_{t|t-1})\right]^\intercal,\\
    \boldsymbol{H}_t\boldsymbol{A}_t &= \boldsymbol{H}_t\boldsymbol{X}_{t|t-1} \\
    &- \left[ \frac{1}{E} \sum_{j=1}^{E}h(\boldsymbol{x}^j_{t|t-1}), \dots, \frac{1}{E} \sum_{j=1}^{E}h(\boldsymbol{x}^j_{t|t-1}) \right]. \nonumber
\end{align*}
$$

The innovation covariance can now be found with

$$
\begin{align*}
    \boldsymbol{S}_t &= \frac{1}{E - 1} (\boldsymbol{H}_t\boldsymbol{A}_t) (\boldsymbol{H}_t\boldsymbol{A}_t)^\intercal + \boldsymbol{R}_t,
\end{align*}
$$

which is then used to compute the Kalman gain as

$$
\begin{align*}
    \boldsymbol{A}_t &= \boldsymbol{X}_{t|t-1} - \frac{1}{E} \sum_{j=1}^{E}\boldsymbol{x}^j_{t|t-1},\\
    \boldsymbol{K}_t &= \frac{1}{E - 1} \boldsymbol{A}_t (\boldsymbol{H}_t\boldsymbol{A}_t)^\intercal \boldsymbol{S}^{-1}_t.
\end{align*}
$$

With this information, the ensemble can be updated to incorporate the new measurement perturbed by stochastic noise: $\epsilon_{y} \sim \mathcal{N}(0, \boldsymbol{R}_t)$ [9]:

$$
\begin{align*}
    \boldsymbol{\tilde{y}}_t &= \left[ \boldsymbol{y}_t + \epsilon^1_y, \dots, \boldsymbol{y}_t + \epsilon^E_y \right], \nonumber \\
    \boldsymbol{X}_{t|t} &= \boldsymbol{X}_{t|t-1} + \boldsymbol{K} (\boldsymbol{\tilde{y}}_{t} - \boldsymbol{H}_t\boldsymbol{X}_{t|t-1}).
\end{align*}
$$

It has been shown that when $\epsilon_{y} \sim \mathcal{N}(0, \boldsymbol{R}_t)$, the measurements are treated as random variables and the ensemble accurately reflects the error covariance of the best state estimate [9].

One of the advantages of this algorithm is the elimination of linearization errors through the use of the nonlinear functions.
While this introduces non-normality into the state uncertainties, it has been shown that the stochastic noise added to the measurements pushes the updated ensemble towards normality, thereby reducing the effects of higher-order moments [10] and improving robustness in nonlinear scenarios.

##### Non-Gaussian Prior

Another source of non-Gaussian uncertainty is from the initial estimate (the prior) itself.
In BIP, our prior is given by a set of demonstrations which indicate where we believe a successful demonstration would lie in the state space.
As we have yet to assimilate any observations of a new interaction, the (unknown) true distribution from which the demonstrations are sampled represents our best initial estimate of what it may be.
However, given that these are real-world demonstrations, they are highly unlikely to be normally distributed.
As such, two options are available in this case: we can either use the demonstrations directly as samples from the non-Gaussian prior distribution or approximate the true distribution with a Gaussian and sample from it.
The latter approach is used by BIP in Eq. 5, however, this comes with its own risk as a poor initial estimate can lead to poor state estimates [11].
Given that the ensemble-based filtering proposed here provides a degree of robustness to non-Gaussian uncertainties, we choose to use samples from the non-Gaussian prior directly in the eBIP algorithm, with the knowledge that the ensemble will be pushed towards normality.
In the event that the number of ensemble members $E$ is greater than the number of available demonstrations, then the density of the true interaction distribution will need to be estimated given the observed demonstrations. This can be accomplished using any density estimation technique, however, we employ a Gaussian mixture model here and denote the resulting algorithm as eBIP$^-$.

##### Computational Performance

Many HRI applications require the use of multiple sensors, which increases the size of the latent space dimension and results in undesirable increases in computation time in the BIP algorithm.
This is due to the necessary covariance matrix operations defined in Eq. 4, which causes BIP to yield an asymptotic computational complexity of approximately $O(n^{3})$ -- with the state of the art lower bounded at approximately $O(n^{2.4})$ -- where $n$ is the state dimension.
However, as eBIP is ensemble-based, we no longer explicitly maintain a covariance matrix; this information is implicitly captured by the ensemble.
As a result, the computational complexity for eBIP is approximately $O(E^2n)$, where $E$ is the ensemble size and $n$ is the state dimension [12].
Since the ensemble size is typically much smaller than the state dimension, this results in a performance increase when compared to BIP.
Furthermore, the formulation presented in this work also obviates the need to explicitly construct the observation matrix $\boldsymbol{H}$.
The creation of the observation matrix introduces an additional overhead for BIP as it must be initialized at each time step due to the phase-dependence, a process which is unnecessary in eBIP.

Ensemble Bayesian Interaction Primitives also benefit from the computational performance-accuracy trade off inherent to all sample-based methods. Inference accuracy can be sacrificed for computational performance by lowering the number of ensemble members when called for. While this is also true for particle filters, they generally scale poorly to higher state dimensions due to sample degeneracy. In particle filtering, ensemble members are re-sampled according to their weight in a scheme known as importance sampling. However, in large state spaces it is likely that only a small number of ensemble members will have high weights, thus eventually causing all members to be re-sampled from only a few. In our proposed method this is not the case, as all members are treated as if they have equal weight, thus lending itself well to high-dimensional state spaces.

## References

[1] Amor, H.B., Neumann, G., Kamthe, S., Kroemer, O. and Peters, J., 2014, May. Interaction primitives for human-robot cooperation tasks. In 2014 IEEE international conference on robotics and automation (ICRA) (pp. 2831-2837). IEEE.

[2] Ijspeert, A.J., Nakanishi, J., Hoffmann, H., Pastor, P. and Schaal, S., 2013. Dynamical movement primitives: learning attractor models for motor behaviors. Neural computation, 25(2), pp.328-373.

[3] Giszter, S.F., Mussa-Ivaldi, F.A. and Bizzi, E., 1993. Convergent force fields organized in the frog's spinal cord. Journal of neuroscience, 13(2), pp.467-491.

[4] Paraschos, A., Daniel, C., Peters, J.R. and Neumann, G., 2013. Probabilistic movement primitives. In Advances in neural information processing systems (pp. 2616-2624).

[5] Maeda, G., Ewerton, M., Lioutikov, R., Amor, H.B., Peters, J. and Neumann, G., 2014, November. Learning interaction for collaborative tasks with probabilistic movement primitives. In 2014 IEEE-RAS International Conference on Humanoid Robots (pp. 527-534). IEEE.

[6] Campbell, J. and Amor, H.B., 2017, October. Bayesian interaction primitives: A slam approach to human-robot interaction. In Conference on Robot Learning (pp. 379-387).

[7] Campbell, J., Stepputtis, S. and Amor, H.B., 2019. Probabilistic Multimodal Modeling for Human-Robot Interaction Tasks. In Robotics: Science and Systems 2019.

[8] Evensen, G., 2003. The ensemble Kalman filter: Theoretical formulation and practical implementation. Ocean dynamics, 53(4), pp.343-367.

[9] Burgers, G., Jan van Leeuwen, P. and Evensen, G., 1998. Analysis scheme in the ensemble Kalman filter. Monthly weather review, 126(6), pp.1719-1724.

[10] Lawson, W.G. and Hansen, J.A., 2004. Implications of stochastic and deterministic filters as ensemble-based data assimilation methods in varying regimes of error growth. Monthly weather review, 132(8), pp.1966-1981.

[11] Haseltine, E.L. and Rawlings, J.B., 2005. Critical evaluation of extended Kalman filtering and moving-horizon estimation. Industrial & engineering chemistry research, 44(8), pp.2451-2460.

[12] Mandel, J., 2006. Efficient implementation of the ensemble Kalman filter. University of Colorado at Denver and Health Sciences Center, Center for Computational Mathematics.